### with this script you can add attributes from raster or polygons to points 

In [43]:
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt

import geopandas
import rasterio
import matplotlib.pyplot as plt
from shapely.geometry import Point

import ee
import geemap
import pandas as pd

In [44]:
# Initialize Earth Engine
ee.Initialize()

##### read points

In [45]:
#esbae = pd.read_csv('/home/sepal-user/module_results/esbae/DRC/DRC_esbae_supervised_class_1522.csv', delimiter=',')
#esbae = pd.read_csv('/home/sepal-user/module_results/esbae/DRC/DRC_all_ceo_1522_clean.csv', delimiter=',')
esbae = pd.read_csv('/home/sepal-user/module_results/esbae/CAFI/CAFI_esbae_ceo_phaseI_merge.csv', delimiter=',', low_memory=False)

all_columns = esbae.columns.tolist()

# Display the list of column names
print(all_columns)

['geometry', 'UniqueID', 'images', 'mon_images', 'bfast_change_date', 'bfast_magnitude', 'bfast_means', 'cusum_change_date', 'cusum_confidence', 'cusum_magnitude', 'red_mean', 'red_sd', 'red_min', 'red_max', 'nir_mean', 'nir_sd', 'nir_min', 'nir_max', 'swir1_mean', 'swir1_sd', 'swir1_min', 'swir1_max', 'swir2_mean', 'swir2_sd', 'swir2_min', 'swir2_max', 'ndfi_mean', 'ndfi_sd', 'ndfi_min', 'ndfi_max', 'brightness_mean', 'brightness_sd', 'brightness_min', 'brightness_max', 'greenness_mean', 'greenness_sd', 'greenness_min', 'greenness_max', 'wetness_mean', 'wetness_sd', 'wetness_min', 'wetness_max', 'bs_slope_mean', 'bs_slope_sd', 'bs_slope_max', 'bs_slope_min', 'ewma_jrc_date', 'ewma_jrc_change', 'ewma_jrc_magnitude', 'mosum_jrc_date', 'mosum_jrc_change', 'mosum_jrc_magnitude', 'cusum_jrc_date', 'cusum_jrc_change', 'cusum_jrc_magnitude', 'ccdc_change_date', 'ccdc_magnitude', 'aspect', 'dw_class_mode', 'dw_tree_prob__max', 'dw_tree_prob__min', 'dw_tree_prob__stdDev', 'dw_tree_prob_mean', 

In [46]:
#drop any columns if needed
#esbae = esbae.drop('index_right', axis=1)

In [47]:
gdf = geopandas.GeoDataFrame(
    esbae, geometry=geopandas.points_from_xy(esbae.LON, esbae.LAT), crs="EPSG:4326"
)

In [48]:
#read GEE points
points_cmr = ee.FeatureCollection('users/faocongo/sbae/sbae_hex16_cmr')
points_drc = ee.FeatureCollection('users/faocongo/sbae/sbae_hex16_drc')
points_eqg = ee.FeatureCollection('users/faocongo/sbae/sbae_hex16_eqg')
points_gab = ee.FeatureCollection('users/faocongo/sbae/sbae_hex16_gab')
points_car = ee.FeatureCollection('users/faocongo/sbae/sbae_hex16_car')

#### add attributes from local shapefile

In [49]:
#poly = "/home/sepal-user/data/drc/RDC_Province_26.shp"
#poly = "/home/sepal-user/data/drc/DRC_gadm2.shp"
poly = "/home/sepal-user/data/other_data/CAFI_tnc_ecoregions.shp"
# Read file using gpd.read_file()
poly_shp = gpd.read_file(poly)
poly_shp.head()

,ECO_ID_U,ECO_CODE,ECO_NAME,ECO_NUM,ECODE_NAME,CLS_CODE,ECO_NOTES,WWF_REALM,WWF_REALM2,WWF_MHTNUM,...,RealmMHT,ER_UPDATE,ER_DATE_U,ER_RATION,SOURCEDATA,Shape__Are,Shape__Len,GlobalID,TropBiome,geometry
0,10085,AT0101,Albertine Rift Montane Forests,1,AT0101. Albertine Rift montane forests,0,None,AT,Afrotropic,1,...,AT1,None,None,None,"Olson, 2001",1.044788e+11,4.618169e+06,{42DD0CDB-F0BD-4800-A5F7-14B72E171211},2,"MULTIPOLYGON (((3231788.497 -744651.154, 32343..."
1,10086,AT0102,Atlantic Equatorial Coastal Forests,2,AT0102. Atlantic Equatorial coastal forests,0,None,AT,Afrotropic,1,...,AT1,None,None,None,"Olson, 2001",1.904370e+11,3.901777e+06,{5976C65D-4B05-43EB-88ED-A37340BC5DBE},1,"MULTIPOLYGON (((1463769.430 -514046.527, 14704..."
2,10087,AT0103,Cameroonian Highlands Forests,3,AT0103. Cameroonian Highlands forests,0,None,AT,Afrotropic,1,...,AT1,None,None,None,"Olson, 2001",3.863439e+10,3.295634e+06,{FE8A64D4-179E-4567-9CBE-760E2A5BE8FB},2,"MULTIPOLYGON (((1206301.417 670801.044, 121015..."
3,10088,AT0104,Central Congolian Lowland Forests,4,AT0104. Central Congolian lowland forests,0,None,AT,Afrotropic,1,...,AT1,None,None,None,"Olson, 2001",4.162152e+11,6.636572e+06,{C2AD3B34-2651-4C44-B72B-589A26C57F14},1,"POLYGON ((2863336.556 -299920.572, 2852158.979..."
4,10091,AT0107,Cross-Sanaga-Bioko Coastal Forests,7,AT0107. Cross-Sanaga-Bioko coastal forests,0,None,AT,Afrotropic,1,...,AT1,None,None,None,"Olson, 2001",5.261769e+10,2.111072e+06,{E1857F6E-2947-405E-B0C5-4827D614EAF4},1,"MULTIPOLYGON (((970579.435 421703.433, 970650...."


In [50]:
poly_crs = poly_shp.crs
print("Current CRS:", poly_crs)

Current CRS: EPSG:3857


In [51]:
poly_shp = poly_shp.to_crs(gdf.crs)

In [52]:
len(esbae)

13115

In [53]:
# Perform the spatial join
joined_data = gpd.sjoin(gdf, poly_shp, how="left", op="within",lsuffix='left', rsuffix='right')

# 'joined_data' now contains the points with additional information from the polygons

# Print or explore the resulting GeoDataFrame
joined_data.head()

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


,geometry,UniqueID,images,mon_images,bfast_change_date,bfast_magnitude,bfast_means,cusum_change_date,cusum_confidence,cusum_magnitude,...,WWF_MHTNAM,RealmMHT,ER_UPDATE,ER_DATE_U,ER_RATION,SOURCEDATA,Shape__Are,Shape__Len,GlobalID,TropBiome
0,POINT (25.60212 -11.69189),1,181,97,2018.589,-738.678406,-1.354781,2016.597,0.111200,45954.212113,...,"Tropical and Subtropical Grasslands, Savannas ...",AT7,None,None,None,"Olson, 2001",1.224692e+12,1.844768e+07,{2F5B243D-96BF-441A-BC7D-453BCA87DF27},2.0
1,POINT (19.01298 6.40494),2,153,90,2016.381,-1332.806396,-1.765601,2020.038,0.318602,53800.048638,...,"Tropical and Subtropical Grasslands, Savannas ...",AT7,None,None,None,"Olson, 2001",7.166044e+11,1.008541e+07,{C312E33F-1D8F-41A8-8C3E-79E7117AD185},2.0
2,POINT (22.79183 7.07841),3,158,93,2016.915,-2419.585449,-2.557598,2020.003,0.116307,48050.450691,...,"Tropical and Subtropical Grasslands, Savannas ...",AT7,None,None,None,"Olson, 2001",9.415629e+11,1.396433e+07,{F7950B56-0AB5-4C79-B651-45D0D1443617},2.0
3,POINT (27.43010 -6.53484),4,129,60,-1.000,0.000000,0.000000,2017.430,0.290449,37411.044881,...,"Tropical and Subtropical Grasslands, Savannas ...",AT7,None,None,None,"Olson, 2001",1.224692e+12,1.844768e+07,{2F5B243D-96BF-441A-BC7D-453BCA87DF27},2.0
4,POINT (22.21215 -9.94098),5,140,79,2018.342,13.038783,-0.582325,2018.518,0.164515,32283.895600,...,"Tropical and Subtropical Grasslands, Savannas ...",AT7,None,None,None,"Olson, 2001",1.224692e+12,1.844768e+07,{2F5B243D-96BF-441A-BC7D-453BCA87DF27},2.0


In [80]:
joined_data['TropBiome'].value_counts(dropna=False)

TropBiome
2.0    8637
1.0    4478
Name: count, dtype: int64

In [55]:
joined_data['TropBiome'] = joined_data['TropBiome'].fillna(2)

In [56]:
list_columns = joined_data.columns.tolist()

# Display the list of column names
print(list_columns)

['geometry', 'UniqueID', 'images', 'mon_images', 'bfast_change_date', 'bfast_magnitude', 'bfast_means', 'cusum_change_date', 'cusum_confidence', 'cusum_magnitude', 'red_mean', 'red_sd', 'red_min', 'red_max', 'nir_mean', 'nir_sd', 'nir_min', 'nir_max', 'swir1_mean', 'swir1_sd', 'swir1_min', 'swir1_max', 'swir2_mean', 'swir2_sd', 'swir2_min', 'swir2_max', 'ndfi_mean', 'ndfi_sd', 'ndfi_min', 'ndfi_max', 'brightness_mean', 'brightness_sd', 'brightness_min', 'brightness_max', 'greenness_mean', 'greenness_sd', 'greenness_min', 'greenness_max', 'wetness_mean', 'wetness_sd', 'wetness_min', 'wetness_max', 'bs_slope_mean', 'bs_slope_sd', 'bs_slope_max', 'bs_slope_min', 'ewma_jrc_date', 'ewma_jrc_change', 'ewma_jrc_magnitude', 'mosum_jrc_date', 'mosum_jrc_change', 'mosum_jrc_magnitude', 'cusum_jrc_date', 'cusum_jrc_change', 'cusum_jrc_magnitude', 'ccdc_change_date', 'ccdc_magnitude', 'aspect', 'dw_class_mode', 'dw_tree_prob__max', 'dw_tree_prob__min', 'dw_tree_prob__stdDev', 'dw_tree_prob_mean', 

In [57]:
#drop any columns if needed
columns_to_drop = ['index_right', 'ECO_ID_U','CLS_CODE', 'ECO_NOTES', 'WWF_REALM', 'WWF_REALM2', 'WWF_MHTNUM', 'WWF_MHTNAM', 'RealmMHT', 'ER_UPDATE', 'ER_DATE_U', 'ER_RATION', 'SOURCEDATA', 'Shape__Are', 'Shape__Len', 'GlobalID']
joined_data = joined_data.drop(columns_to_drop, axis=1)

In [58]:
# Rename columns
#column_mapping = {
#    'NAME_1': 'Province',
#    'NAME_2': 'Territoire',
#    'Unnamed: 0':'Index'
#}

# Use the rename() method to rename columns
#joined_data.rename(columns=column_mapping, inplace=True)

In [59]:
joined_columns = joined_data.columns.tolist()

# Display the list of column names
print(joined_columns)

['geometry', 'UniqueID', 'images', 'mon_images', 'bfast_change_date', 'bfast_magnitude', 'bfast_means', 'cusum_change_date', 'cusum_confidence', 'cusum_magnitude', 'red_mean', 'red_sd', 'red_min', 'red_max', 'nir_mean', 'nir_sd', 'nir_min', 'nir_max', 'swir1_mean', 'swir1_sd', 'swir1_min', 'swir1_max', 'swir2_mean', 'swir2_sd', 'swir2_min', 'swir2_max', 'ndfi_mean', 'ndfi_sd', 'ndfi_min', 'ndfi_max', 'brightness_mean', 'brightness_sd', 'brightness_min', 'brightness_max', 'greenness_mean', 'greenness_sd', 'greenness_min', 'greenness_max', 'wetness_mean', 'wetness_sd', 'wetness_min', 'wetness_max', 'bs_slope_mean', 'bs_slope_sd', 'bs_slope_max', 'bs_slope_min', 'ewma_jrc_date', 'ewma_jrc_change', 'ewma_jrc_magnitude', 'mosum_jrc_date', 'mosum_jrc_change', 'mosum_jrc_magnitude', 'cusum_jrc_date', 'cusum_jrc_change', 'cusum_jrc_magnitude', 'ccdc_change_date', 'ccdc_magnitude', 'aspect', 'dw_class_mode', 'dw_tree_prob__max', 'dw_tree_prob__min', 'dw_tree_prob__stdDev', 'dw_tree_prob_mean', 

In [60]:
coord_list = [(x, y) for x, y in zip(joined_data["geometry"].x, joined_data["geometry"].y)]

#### get attributes from local raster

In [61]:
LC_map = "/home/sepal-user/data/lc_map/CAFI_LC_2015_19_wgs84.tif"

In [62]:
LC_src = rasterio.open(LC_map)

In [63]:
frag_map = "/home/sepal-user/data/other_data/cafi_frag_wgs84.tif"

In [64]:
frag_src = rasterio.open(frag_map)

In [65]:
treecov = "/home/sepal-user/data/other_data/treecover_2015_1ha_wgs84.tif"

In [66]:
tree_src = rasterio.open(treecov)

In [67]:
biomass = "/home/sepal-user/data/other_data/ESA_Biomass_2010.tif"

In [68]:
biomass_src =  rasterio.open(biomass)

In [69]:
joined_data["CAFI_LC_2015"] = [x for x in LC_src.sample(coord_list)]
joined_data["CAFI_LC_2015"] = joined_data["CAFI_LC_2015"].str[0]
joined_data.head()

,geometry,UniqueID,images,mon_images,bfast_change_date,bfast_magnitude,bfast_means,cusum_change_date,cusum_confidence,cusum_magnitude,...,Degall,Stable,NF,sampling,ECO_CODE,ECO_NAME,ECO_NUM,ECODE_NAME,TropBiome,CAFI_LC_2015
0,POINT (25.60212 -11.69189),1,181,97,2018.589,-738.678406,-1.354781,2016.597,0.111200,45954.212113,...,1,0,0,str_random,AT0704,Central Zambezian Miombo Woodlands,4.0,AT0704. Central Zambezian Miombo woodlands,2.0,2
1,POINT (19.01298 6.40494),2,153,90,2016.381,-1332.806396,-1.765601,2020.038,0.318602,53800.048638,...,1,0,0,str_random,AT0712,Northern Congolian Forest-Savanna Mosaic,12.0,AT0712. Northern Congolian forest-savanna mosaic,2.0,9
2,POINT (22.79183 7.07841),3,158,93,2016.915,-2419.585449,-2.557598,2020.003,0.116307,48050.450691,...,1,0,0,str_random,AT0705,East Sudanian Savanna,5.0,AT0705. East Sudanian savanna,2.0,11
3,POINT (27.43010 -6.53484),4,129,60,-1.000,0.000000,0.000000,2017.430,0.290449,37411.044881,...,0,1,0,str_random,AT0704,Central Zambezian Miombo Woodlands,4.0,AT0704. Central Zambezian Miombo woodlands,2.0,11
4,POINT (22.21215 -9.94098),5,140,79,2018.342,13.038783,-0.582325,2018.518,0.164515,32283.895600,...,0,1,0,str_random,AT0704,Central Zambezian Miombo Woodlands,4.0,AT0704. Central Zambezian Miombo woodlands,2.0,2


In [70]:
joined_data['CAFI_LC_2015'].value_counts(dropna=False)

CAFI_LC_2015
1     3345
4     1560
12    1150
3     1137
15     959
19     633
11     589
6      576
13     556
7      530
5      482
2      427
8      383
9      267
16     265
14     204
17      35
18      16
0        1
Name: count, dtype: int64

In [71]:
joined_data["Frag_2015"] = [x for x in frag_src.sample(coord_list)]
joined_data["Frag_2015"] = joined_data["Frag_2015"].str[0]
joined_data.head()

,geometry,UniqueID,images,mon_images,bfast_change_date,bfast_magnitude,bfast_means,cusum_change_date,cusum_confidence,cusum_magnitude,...,Stable,NF,sampling,ECO_CODE,ECO_NAME,ECO_NUM,ECODE_NAME,TropBiome,CAFI_LC_2015,Frag_2015
0,POINT (25.60212 -11.69189),1,181,97,2018.589,-738.678406,-1.354781,2016.597,0.111200,45954.212113,...,0,0,str_random,AT0704,Central Zambezian Miombo Woodlands,4.0,AT0704. Central Zambezian Miombo woodlands,2.0,2,5
1,POINT (19.01298 6.40494),2,153,90,2016.381,-1332.806396,-1.765601,2020.038,0.318602,53800.048638,...,0,0,str_random,AT0712,Northern Congolian Forest-Savanna Mosaic,12.0,AT0712. Northern Congolian forest-savanna mosaic,2.0,9,3
2,POINT (22.79183 7.07841),3,158,93,2016.915,-2419.585449,-2.557598,2020.003,0.116307,48050.450691,...,0,0,str_random,AT0705,East Sudanian Savanna,5.0,AT0705. East Sudanian savanna,2.0,11,2
3,POINT (27.43010 -6.53484),4,129,60,-1.000,0.000000,0.000000,2017.430,0.290449,37411.044881,...,1,0,str_random,AT0704,Central Zambezian Miombo Woodlands,4.0,AT0704. Central Zambezian Miombo woodlands,2.0,11,2
4,POINT (22.21215 -9.94098),5,140,79,2018.342,13.038783,-0.582325,2018.518,0.164515,32283.895600,...,1,0,str_random,AT0704,Central Zambezian Miombo Woodlands,4.0,AT0704. Central Zambezian Miombo woodlands,2.0,2,3


In [72]:
joined_data['Frag_2015'].value_counts(dropna=False)

Frag_2015
2    5394
0    2658
5    2256
3    1695
1     853
4     259
Name: count, dtype: int64

In [73]:
joined_data["Treecov_2015"] = [x for x in tree_src.sample(coord_list)]
joined_data["Treecov_2015"] = joined_data["Treecov_2015"].str[0]
joined_data.head()

,geometry,UniqueID,images,mon_images,bfast_change_date,bfast_magnitude,bfast_means,cusum_change_date,cusum_confidence,cusum_magnitude,...,NF,sampling,ECO_CODE,ECO_NAME,ECO_NUM,ECODE_NAME,TropBiome,CAFI_LC_2015,Frag_2015,Treecov_2015
0,POINT (25.60212 -11.69189),1,181,97,2018.589,-738.678406,-1.354781,2016.597,0.111200,45954.212113,...,0,str_random,AT0704,Central Zambezian Miombo Woodlands,4.0,AT0704. Central Zambezian Miombo woodlands,2.0,2,5,52
1,POINT (19.01298 6.40494),2,153,90,2016.381,-1332.806396,-1.765601,2020.038,0.318602,53800.048638,...,0,str_random,AT0712,Northern Congolian Forest-Savanna Mosaic,12.0,AT0712. Northern Congolian forest-savanna mosaic,2.0,9,3,54
2,POINT (22.79183 7.07841),3,158,93,2016.915,-2419.585449,-2.557598,2020.003,0.116307,48050.450691,...,0,str_random,AT0705,East Sudanian Savanna,5.0,AT0705. East Sudanian savanna,2.0,11,2,33
3,POINT (27.43010 -6.53484),4,129,60,-1.000,0.000000,0.000000,2017.430,0.290449,37411.044881,...,0,str_random,AT0704,Central Zambezian Miombo Woodlands,4.0,AT0704. Central Zambezian Miombo woodlands,2.0,11,2,28
4,POINT (22.21215 -9.94098),5,140,79,2018.342,13.038783,-0.582325,2018.518,0.164515,32283.895600,...,0,str_random,AT0704,Central Zambezian Miombo Woodlands,4.0,AT0704. Central Zambezian Miombo woodlands,2.0,2,3,51


In [74]:
joined_data['Treecov_2015'].value_counts(dropna=False)

Treecov_2015
 20     243
 21     236
 19     235
 18     234
 23     216
       ... 
 7       13
 97       5
-128      3
 6        3
 98       1
Name: count, Length: 94, dtype: int64

In [75]:
joined_data["Biomass_2010"] = [x for x in biomass_src.sample(coord_list)]
joined_data["Biomass_2010"] = joined_data["Biomass_2010"].str[0]
joined_data.head()

,geometry,UniqueID,images,mon_images,bfast_change_date,bfast_magnitude,bfast_means,cusum_change_date,cusum_confidence,cusum_magnitude,...,sampling,ECO_CODE,ECO_NAME,ECO_NUM,ECODE_NAME,TropBiome,CAFI_LC_2015,Frag_2015,Treecov_2015,Biomass_2010
0,POINT (25.60212 -11.69189),1,181,97,2018.589,-738.678406,-1.354781,2016.597,0.111200,45954.212113,...,str_random,AT0704,Central Zambezian Miombo Woodlands,4.0,AT0704. Central Zambezian Miombo woodlands,2.0,2,5,52,94
1,POINT (19.01298 6.40494),2,153,90,2016.381,-1332.806396,-1.765601,2020.038,0.318602,53800.048638,...,str_random,AT0712,Northern Congolian Forest-Savanna Mosaic,12.0,AT0712. Northern Congolian forest-savanna mosaic,2.0,9,3,54,194
2,POINT (22.79183 7.07841),3,158,93,2016.915,-2419.585449,-2.557598,2020.003,0.116307,48050.450691,...,str_random,AT0705,East Sudanian Savanna,5.0,AT0705. East Sudanian savanna,2.0,11,2,33,9
3,POINT (27.43010 -6.53484),4,129,60,-1.000,0.000000,0.000000,2017.430,0.290449,37411.044881,...,str_random,AT0704,Central Zambezian Miombo Woodlands,4.0,AT0704. Central Zambezian Miombo woodlands,2.0,11,2,28,52
4,POINT (22.21215 -9.94098),5,140,79,2018.342,13.038783,-0.582325,2018.518,0.164515,32283.895600,...,str_random,AT0704,Central Zambezian Miombo Woodlands,4.0,AT0704. Central Zambezian Miombo woodlands,2.0,2,3,51,87


In [76]:
joined_data['Biomass_2010'].value_counts(dropna=False)

Biomass_2010
0      1379
1       171
2       152
3       118
4       107
       ... 
461       1
622       1
659       1
617       1
548       1
Name: count, Length: 651, dtype: int64

In [77]:
joined_columns = joined_data.columns.tolist()

# Display the list of column names
print(joined_columns)

['geometry', 'UniqueID', 'images', 'mon_images', 'bfast_change_date', 'bfast_magnitude', 'bfast_means', 'cusum_change_date', 'cusum_confidence', 'cusum_magnitude', 'red_mean', 'red_sd', 'red_min', 'red_max', 'nir_mean', 'nir_sd', 'nir_min', 'nir_max', 'swir1_mean', 'swir1_sd', 'swir1_min', 'swir1_max', 'swir2_mean', 'swir2_sd', 'swir2_min', 'swir2_max', 'ndfi_mean', 'ndfi_sd', 'ndfi_min', 'ndfi_max', 'brightness_mean', 'brightness_sd', 'brightness_min', 'brightness_max', 'greenness_mean', 'greenness_sd', 'greenness_min', 'greenness_max', 'wetness_mean', 'wetness_sd', 'wetness_min', 'wetness_max', 'bs_slope_mean', 'bs_slope_sd', 'bs_slope_max', 'bs_slope_min', 'ewma_jrc_date', 'ewma_jrc_change', 'ewma_jrc_magnitude', 'mosum_jrc_date', 'mosum_jrc_change', 'mosum_jrc_magnitude', 'cusum_jrc_date', 'cusum_jrc_change', 'cusum_jrc_magnitude', 'ccdc_change_date', 'ccdc_magnitude', 'aspect', 'dw_class_mode', 'dw_tree_prob__max', 'dw_tree_prob__min', 'dw_tree_prob__stdDev', 'dw_tree_prob_mean', 

In [78]:
joined_data.head()

,geometry,UniqueID,images,mon_images,bfast_change_date,bfast_magnitude,bfast_means,cusum_change_date,cusum_confidence,cusum_magnitude,...,sampling,ECO_CODE,ECO_NAME,ECO_NUM,ECODE_NAME,TropBiome,CAFI_LC_2015,Frag_2015,Treecov_2015,Biomass_2010
0,POINT (25.60212 -11.69189),1,181,97,2018.589,-738.678406,-1.354781,2016.597,0.111200,45954.212113,...,str_random,AT0704,Central Zambezian Miombo Woodlands,4.0,AT0704. Central Zambezian Miombo woodlands,2.0,2,5,52,94
1,POINT (19.01298 6.40494),2,153,90,2016.381,-1332.806396,-1.765601,2020.038,0.318602,53800.048638,...,str_random,AT0712,Northern Congolian Forest-Savanna Mosaic,12.0,AT0712. Northern Congolian forest-savanna mosaic,2.0,9,3,54,194
2,POINT (22.79183 7.07841),3,158,93,2016.915,-2419.585449,-2.557598,2020.003,0.116307,48050.450691,...,str_random,AT0705,East Sudanian Savanna,5.0,AT0705. East Sudanian savanna,2.0,11,2,33,9
3,POINT (27.43010 -6.53484),4,129,60,-1.000,0.000000,0.000000,2017.430,0.290449,37411.044881,...,str_random,AT0704,Central Zambezian Miombo Woodlands,4.0,AT0704. Central Zambezian Miombo woodlands,2.0,11,2,28,52
4,POINT (22.21215 -9.94098),5,140,79,2018.342,13.038783,-0.582325,2018.518,0.164515,32283.895600,...,str_random,AT0704,Central Zambezian Miombo Woodlands,4.0,AT0704. Central Zambezian Miombo woodlands,2.0,2,3,51,87


#### export

In [79]:
out_file =  '/home/sepal-user/module_results/esbae/CAFI/CAFI_esbae_ceo_phaseI_att.csv'
joined_data.to_csv(out_file,index=False)